# Deep Q Learning - Lunar Lander

In this assignment, we train an agent to land a lunar lander safely on a landing pag on the surface of the moon. 

## Import Packages

In [ ]:
import time
from collections import deque, namedtuple

import gym
import numpy as np
import PIL.Image
import tensorflow as tf

from pyvirtualdisplay import Display
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import MSE
from tensorflow.keras.optimizers import Adam

In [ ]:
# # Set up a virtual display to render the Lunar Lander environment.
# Display(visible=0, size=(840, 480)).start();

Package descritions:
- deque - data structure and memory descriptions
- namedtuple - stores experience tuple
- gym - collection of enviornment to test reifn algo
- PIL.Image & pyvirtualdisplay - needed to render lunar lander enviorment

## Hyperparameters

In [ ]:
MEMORY_SIZE = 100_000     # size of memory buffer
GAMMA = 0.995             # discount factor
ALPHA = 1e-3              # learning rate  
NUM_STEPS_FOR_UPDATE = 4  # perform a learning update every C time steps

## Lunar Lander Enviornment
- We use OpenAI's gym library that provides a wide variety of enviornments for reinf learning. An enviornment represents a problem or a task to be solved.
- The landing flag is designated by 2 flag poles and it is always at coordinates (0,0) but the lunar lander is allowed to land outside of the landing pad.
- The lander starts at the top center of the enviornment with a random initial force appolied to its centre of mass and also it has infinite fuel. The enviornment is considered solved on getting 200 points.

## Action Space
The agent has 4 discrete actions available:
1. Do nothing (corresponding value = 0)
2. Fire right engine (corresponding value = 1)
3. Fire main engine (corresponding value = 2)
4. Fire left engine (corresponding value = 3)

## Observation Space
The agents observation space consists of a state vector with 8 variables:
* Its $(x,y)$ coordinates. The landing pad is always at coordinates $(0,0)$.
* Its linear velocities $(\dot x,\dot y)$.
* Its angle $\theta$.
* Its angular velocity $\dot \theta$.
* Two booleans, $l$ and $r$, that represent whether each leg is in contact with the ground or not.

## Rewards
The lunar lander enviornment has the following reward system:
* Landing on the landing pad and coming to rest is about 100-140 points.
* If the lander moves away from the landing pad, it loses reward. 
* If the lander crashes, it receives -100 points.
* If the lander comes to rest, it receives +100 points.
* Each leg with ground contact is +10 points.
* Firing the main engine is -0.3 points each frame.
* Firing the side engine is -0.03 points each frame.

## Episode Termination
An episode ends if:
* The lunar lander crashes (i.e if the body of the lunar lander comes in contact with the surface of the moon).
* The lander's $x$-coordinate is greater than 1.

## Load the enviornment
We use the gym library to initiate the lunar lander enviornment. Once we load the enviornment, we rest its initial state. We then render the frame

In [ ]:
env = gym.make('LunarLander-v2')
